In [1]:
#Python Implementation of the Hartree Fock Method
#Procedures listed in the code follow as described in Modern Quantum Chemistry: Introduction to Advanced Electronic Structure Theory, By Attila Szabo and Neil S. Ostlund

import math
import numpy as np
from molecule import atom
from molecule import vector
from molecule import guassian
from molecule import molecule
import integrals

#print(g3.constant * pow((math.pi/g3.orbitalExponet), 3/2))	

#Step 1
#Specify Molecules, Nuclear Coordinates, and Charge of the nucli Number of Electrons,

#generate an h2 atom with a distance of 1.4 AU to compare with Szabo pg. 175
h2 = molecule()
h2.addAtom(atom(vector(), 1, 1))
h2.addAtom(atom(vector(.5,.5, 1.208304597), 1, 1))
#h2.addAtom(atom(vector(5,0,0),1,1))
h2.addBasis("STO-3G")

print(len(h2.atomData[1].basisSet.contractedGuassians))

#and a basis set, for now will add default basis of STO-3G for hydrogen to all atoms


SyntaxError: invalid syntax (molecule.py, line 209)

In [ ]:

#Step 2
#Calculate Integrals
#Overlap, KE, Nuclear Attraaction, Electronb 
S = integrals.overlap(h2)
print(np.matrix(S))

In [ ]:
#Step 3
#Dignolize Overlap Matrix and Obtain transformation matrix X


In [ ]:
#Step 4
#Obtain guess Density matrix, P

In [ ]:
#Step 5 
#Calculate Contraction of Density Matrix with Electron Repulsion, G Matrix

In [ ]:
#Step 6
#Calculate Fock matrix, core matrix + G matrix

In [ ]:
#Step 7
#Using X diagnolize Fock Matrix

In [ ]:
#Step 8
#Diagnolize F' to get C' and E'

In [ ]:
#Step 9
#Calculate C from C'

In [ ]:
#Step 10
#From P for next iteration of HF Procedure

In [ ]:
#Step 11
#Determine if current P is close "enough" to previos P, 
#	if no iterate again start from step 5
#	if yes, then stop iterating, HF procedure completed, and use final C matrix to compute physical properties
